# Model conversion to GGUF notebook

In [ ]:
# Start time
!date

Sat May  3 06:11:07 PM UTC 2025


In [ ]:
trainset = "1746028477"
checkpoint = "300"

## Setup

In [ ]:
from getpass import getpass

def get_secret(prompt, secret_name, secret_input=True):
  try:
    from google.colab import userdata
    result = userdata.get(secret_name)
    assert result is not None
  except:
    if secret_input:
      result = getpass(prompt)
    else:
      result = input(prompt)
  return result


In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth
    !pip install -U transformers

from unsloth import FastLanguageModel  # Load unsloth ASAP

## Download pre-trained checkpoint

In [ ]:
import os

if (
    (not os.path.exists(os.path.expanduser("~/.ssh/id_rsa")))
    or (not os.path.exists(os.path.expanduser("~/.ssh/known_hosts")))
):
  SSHKEY = get_secret('Result pusher SSH key: ', 'TFM_SSH_PUSHER_KEY')

  !mkdir ~/.ssh

  # Read locally with `cat ~/.ssh/result-pusher|tr '\n' '$';echo`
  with open(os.path.expanduser("~/.ssh/id_rsa"), 'wt') as f:
    f.write(SSHKEY.replace('$', '\n'))

  !chmod 0600 ~/.ssh/id_rsa
  !ssh-keygen -y -f ~/.ssh/id_rsa > ~/.ssh/id_rsa.pub
  !chmod 0600 ~/.ssh/id_rsa.pub

  # This won't copy the client key (not needed), but it will initialize the server's on the client
  !ssh-copy-id -i ~/.ssh/id_rsa -o StrictHostKeyChecking=accept-new result-pusher@kb.tfm.codigoparallevar.com

  del SSHKEY

/usr/bin/ssh-copy-id: INFO: Source of key(s) to be installed: "/root/.ssh/id_rsa.pub"
/usr/bin/ssh-copy-id: INFO: attempting to log in with the new key(s), to filter out any that are already installed

/usr/bin/ssh-copy-id: WARNING: All keys were skipped because they already exist on the remote system.
		(if you think this is a mistake, you may want to use -f option)



In [ ]:
!rsync -HPrz --mkpath \
  result-pusher@kb.tfm.codigoparallevar.com:fine-tuning/fine-tuned/peft-kbs-summary-training-"$trainset"/checkpoint-"$checkpoint"/ \
    fine-tune

receiving incremental file list
created 1 directory for fine-tune
README.md
          5,104 100%    4.87MB/s    0:00:00 (xfr#1, to-chk=12/14)
adapter_config.json
            860 100%  839.84kB/s    0:00:00 (xfr#2, to-chk=11/14)
adapter_model.safetensors
    262,219,392 100%   10.80MB/s    0:00:23 (xfr#3, to-chk=10/14)
merges.txt
        916,646 100%    1.47MB/s    0:00:00 (xfr#4, to-chk=9/14)
optimizer.pt
    133,785,108 100%   10.75MB/s    0:00:11 (xfr#5, to-chk=8/14)
rng_state.pth
         14,244 100%   24.23kB/s    0:00:00 (xfr#6, to-chk=7/14)
scheduler.pt
          1,064 100%    1.81kB/s    0:00:00 (xfr#7, to-chk=6/14)
special_tokens_map.json
            456 100%    0.78kB/s    0:00:00 (xfr#8, to-chk=5/14)
tokenizer.json
      7,153,264 100%    8.92MB/s    0:00:00 (xfr#9, to-chk=4/14)
tokenizer_config.json
         17,987 100%   22.96kB/s    0:00:00 (xfr#10, to-chk=3/14)
trainer_state.json
          3,599 100%    4.59kB/s    0:00:00 (xfr#11, to-chk=2/14)
training_args.bin
         

## Load pre-trained model

In [ ]:
from peft import PeftModel
import torch

device_map = {"": 0}


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-4",
    max_seq_length = 16384, # max_seq_length,
    load_in_4bit = True, # load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

ft_model = PeftModel.from_pretrained(model, "fine-tune",torch_dtype=torch.float16,is_trainable=False)

==((====))==  Unsloth 2025.4.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/160k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.39G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/18.0k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/917k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.15M [00:00<?, ?B/s]

## Save to GGUF

In [ ]:
outname = f"phi-4-{trainset}-cp-{checkpoint}"

In [12]:
ft_model.save_pretrained_gguf(outname, tokenizer, quantization_method = [ "f16" ]) # , "q4_k_m"])

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 37.37 out of 52.96 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 40/40 [00:48<00:00,  1.20s/it]


Unsloth: Saving tokenizer... Done.
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['f16', 'q4_k_m', 'q6_k'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: [1] Converting model at phi-4-1746474294-cp-855 into f16 GGUF format.
The output location will be /content/phi-4-1746474294-cp-855/unsloth.F16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: phi-4-1746474294-cp-855
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Set model parameters
INFO:hf-to-gguf:gguf: context length = 16384
INFO:hf-to-gguf:gguf: embedding length = 5120
INFO:hf-to-gguf:gguf: feed forward length = 17920
INFO:hf-to-gguf:gguf: head count = 40
INFO:hf-to-g

## Upload data back

In [13]:
rsyncto="result-pusher@kb.tfm.codigoparallevar.com:fine-tuning/fine-tuned/peft-kbs-summary-training-" + trainset + "/checkpoint-" + checkpoint + "/loadable"

In [14]:
!rsync -HPrz --mkpath "$outname"/ "$rsyncto"

sending incremental file list
config.json
            818 100%    0.00kB/s    0:00:00 (xfr#1, to-chk=16/18)
generation_config.json
            170 100%  166.02kB/s    0:00:00 (xfr#2, to-chk=15/18)
merges.txt
        916,646 100%   72.85MB/s    0:00:00 (xfr#3, to-chk=14/18)
model-00001-of-00006.safetensors
  4,933,658,528 100%   13.39MB/s    0:05:51 (xfr#4, to-chk=13/18)
model-00002-of-00006.safetensors
  4,954,693,112 100%   13.45MB/s    0:05:51 (xfr#5, to-chk=12/18)
model-00003-of-00006.safetensors
  4,902,243,992 100%   13.44MB/s    0:05:47 (xfr#6, to-chk=11/18)
model-00004-of-00006.safetensors
  4,954,672,440 100%   13.40MB/s    0:05:52 (xfr#7, to-chk=10/18)
model-00005-of-00006.safetensors
  4,954,672,432 100%   13.35MB/s    0:05:53 (xfr#8, to-chk=9/18)
model-00006-of-00006.safetensors
  4,619,116,224 100%   13.28MB/s    0:05:31 (xfr#9, to-chk=8/18)
model.safetensors.index.json
         29,894 100%  789.01kB/s    0:00:00 (xfr#10, to-chk=7/18)
special_tokens_map.json
            570

## Completed

In [15]:
!date

Tue May  6 12:22:18 PM UTC 2025
